# Street noise quietness notebook

## 0. Initialization

### 0.1. Load required libraries

In [ ]:
import os
import topogenesis as tg
import pyvista as pv
import trimesh as tm
import numpy as np
import networkx as nx
import scipy as sp
np.random.seed(0)

### 0.2 Load availability lattice and meshes

In [ ]:
# loading the lattice from csv
lattice_path = os.path.relpath('../data/solar_envelope_324.csv')
avail_lattice = tg.lattice_from_csv(lattice_path)
init_avail_lattice = tg.to_lattice(np.copy(avail_lattice), avail_lattice)

#Load context mesh from csv
context_path = os.path.relpath("../data/immediate_context.obj")
street_50_path = os.path.relpath("../data/street_noise_50db_3.obj") 
street_70_path = os.path.relpath("../data/street_noise_70db_4.obj") 

# load the mesh from file
context_mesh = tm.load(context_path)
street_50_mesh = tm.load(street_50_path)
street_70_mesh = tm.load(street_70_path)

# construct full latice
full_lattice = avail_lattice * 0 + 1 

### 0.3 visualize

In [ ]:
pv.set_plot_theme("document")

# convert mesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

# Visualize the mesh using pyvista plotter
#######

# initiating the plotter
p = pv.Plotter(notebook=True)

# fast visualization of the lattice
avail_lattice.fast_vis(p)

# adding the meshes
p.add_mesh(tri_to_pv(context_mesh), opacity=0.1, style='wireframe')
p.add_mesh(tri_to_pv(street_50_mesh), color="ffd571", lighting = False, style='wireframe', line_width = 5)
p.add_mesh(tri_to_pv(street_70_mesh), color="ff7b7b", lighting = False, style='wireframe', line_width = 5)


# plotting
cpos = [(785.8704805788776, 708.4540755788776, 741.8613927288776),
 (65.08283250000001, -12.333572500000002, 21.07374465),
 (0.0, 0.0, 1.0)]
p.camera_position = cpos
p.window_size = 2000, 2000
p.show(use_ipyvtk=True)
# p.screenshot("noise_1")
print(p.camera_position)

### 1.1. 1st street - Distances between voxels and mesh

In [ ]:
#Get voxel centers as points
vox_centroids = full_lattice.centroids
vox_ind = np.array(np.where(full_lattice==1)).T

# Noise level in first street
noise_base_1 = 50.0

#calculate closest distance from each voxel to point
dist_latice = avail_lattice * 0.0

for cen, ind in zip (vox_centroids, vox_ind):

    closest, distance, triangle_id = tm.proximity.closest_point(street_50_mesh, np.array([cen]))
    distance=np.round_(distance, 2)
    distance+= 0.01
    dist_latice[tuple(ind)] = distance

# computing the noise lattice from dist lattice
noise_latice_50 = noise_base_1 - 20 * np.log10(dist_latice) - 8

# initializing the sum lattice of noise
sum_noise_lats = avail_lattice * 0.0

# summing
sum_noise_lats += np.power(10, noise_latice_50 / 10.0)

### 1.2. 2nd street - Distances between voxels and mesh

In [ ]:
#Get voxel centers as points / done with "_2" for troubleshooting
vox_centroids_2 = full_lattice.centroids
vox_ind_2 = np.array(np.where(full_lattice==1)).T

# Noise level in second street
noise_base_2 = 70.0

#calculate closest distance from each voxel to point
dist_latice_2 = avail_lattice * 0.0

for cen_2, ind_2 in zip (vox_centroids_2, vox_ind_2):

    closest, distance_2, triangle_id = tm.proximity.closest_point(street_70_mesh, np.array([cen_2]))
    distance_2+= 0.01
    distance_2=np.round_(distance_2, 2)
    dist_latice_2[tuple(ind_2)] = distance_2

# computing the noise lattice from dist lattice
noise_latice_70 = noise_base_2 - 20 * np.log10(dist_latice_2) -8

# summing
sum_noise_lats += np.power(10, noise_latice_70 / 10.0)

In [ ]:
# computing the final aggregation
agg_noise_lats = 10 * np.log10(sum_noise_lats)

### 1.3. Visulization

In [ ]:
# initiating the plotter
p = pv.Plotter(notebook=True)

# adding the meshes
p.add_mesh(tri_to_pv(context_mesh), opacity=0.1, style='wireframe')
p.add_mesh(tri_to_pv(street_50_mesh), color="ffd571", lighting = False, style='wireframe', line_width = 5)
p.add_mesh(tri_to_pv(street_70_mesh), color="ff7b7b", lighting = False, style='wireframe', line_width = 5)

vis_lattice = agg_noise_lats * avail_lattice

# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid dimensions: shape because we want to inject our values
grid.dimensions = vis_lattice.shape
# The bottom left corner of the data set
grid.origin = vis_lattice.minbound
# These are the cell sizes along each axis
grid.spacing = vis_lattice.unit

# Add the data values to the cell data
grid.point_arrays["Street noise in dB"] = vis_lattice.flatten(order="F")  # Flatten the Lattice

# adding the volume
opacity = np.array([0,0.6,0.6,0.6,0.6,0.6,0.6])
p.add_volume(grid, cmap="coolwarm" , clim=[20, 40], opacity=opacity, shade=False)

# plotting
cpos = [(785.8704805788776, 708.4540755788776, 741.8613927288776),
 (65.08283250000001, -12.333572500000002, 21.07374465),
 (0.0, 0.0, 1.0)]
p.camera_position = cpos
p.window_size = 2000, 2000
p.show(use_ipyvtk=True)
# p.screenshot("noise_2")
print(p.camera_position)

### 2.1. Mapping the values


In [ ]:
# mapping the values from (0, max) to (1, 0)

max_noise = np.max(agg_noise_lats)
quietness_lattice = (1- agg_noise_lats / max_noise) * avail_lattice

### 2.2. Visualize noise value field

In [ ]:
# convert mesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

# load the mesh from file
context_path = os.path.relpath('../data/immediate_context.obj')
context_mesh = tm.load(context_path)

# initiating the plotter
p = pv.Plotter(notebook=True)

p.add_mesh(tri_to_pv(context_mesh), opacity=0.1, style='wireframe')
p.add_mesh(tri_to_pv(street_50_mesh), color="ffd571", lighting = False, style='wireframe', line_width = 5)
p.add_mesh(tri_to_pv(street_70_mesh), color="ff7b7b", lighting = False, style='wireframe', line_width = 5)

# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid dimensions: shape because we want to inject our values
grid.dimensions = avail_lattice.shape
# The bottom left corner of the data set
grid.origin = avail_lattice.minbound
# These are the cell sizes along each axis
grid.spacing = avail_lattice.unit

# Add the data values to the cell data
grid.point_arrays["Quietness from street noise"] = quietness_lattice.flatten(order="F")  # Flatten the Lattice
    
# adding the volume
opacity = np.array([0.0,0.6,0.6,0.6,0.6,0.6,0.6])
p.add_volume(grid, cmap="coolwarm", clim=[0.5, 1.0] ,opacity=opacity)

# plotting
cpos = [(785.8704805788776, 708.4540755788776, 741.8613927288776),
 (65.08283250000001, -12.333572500000002, 21.07374465),
 (0.0, 0.0, 1.0)]
p.camera_position = cpos
p.window_size = 2000, 2000
p.show(use_ipyvtk=True)
# p.screenshot("noise_3")
print(p.camera_position)

### 3.1. Save Quietness from street noise lattice to CSV

In [ ]:
# save the interest voxel access lattice

csv_path = os.path.relpath('../data/project_data/quietness_from_street_noise_324.csv')
quietness_lattice.to_csv(csv_path)